# Colab Quickstart

In [1]:
!git clone --recursive https://github.com/lucap02/aml-2025-mistake-detection.git code

Cloning into 'code'...
remote: Enumerating objects: 452, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 452 (delta 11), reused 14 (delta 11), pack-reused 428 (from 1)
Receiving objects: 100% (452/452), 116.66 KiB | 1003.00 KiB/s, done.
Resolving deltas: 100% (292/292), done.
Submodule 'annotations' (https://github.com/CaptainCook4D/annotations) registered for path 'annotations'
Cloning into '/content/code/annotations'...
remote: Enumerating objects: 152, done.        
remote: Counting objects: 100% (152/152), done.        
remote: Compressing objects: 100% (98/98), done.        
remote: Total 152 (delta 75), reused 108 (delta 46), pack-reused 0 (from 0)        
Receiving objects: 100% (152/152), 793.14 KiB | 3.30 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Submodule path 'annotations': checked out '0e9a108be2cbcbcbd592e7418c0ab9c16232d27a'


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
"""Setup features on Google Colab.
- Mounts Google Drive (if available)
- Extracts features.zip that contains nested omnivore.zip / slowfast.zip (and any other *.zip)
- Moves .npz files into data/video/{omnivore,slowfast} and data/audio
"""

import os
import shutil
import zipfile
from pathlib import Path
from typing import List

from tqdm import tqdm


def mount_google_drive() -> bool:
    try:
        from google.colab import drive  # type: ignore
        drive.mount("/content/drive")
        print("✓ Google Drive mounted at /content/drive")
        return True
    except Exception:
        print("⚠ Not running on Colab; skipping drive mount")
        return False


def extract_zip(zip_path: str, destination: str) -> Path:
    destination_path = Path(destination)
    destination_path.mkdir(parents=True, exist_ok=True)

    with zipfile.ZipFile(zip_path, "r") as zf:
        members = zf.namelist()
        print(f"📦 Extracting {zip_path} -> {destination} ({len(members)} entries)")
        for member in tqdm(members, desc="Extract", unit="file"):
            zf.extract(member, destination)
    return destination_path


def extract_inner_zips(root: Path) -> List[Path]:
    extracted_dirs = []
    for zip_file in root.rglob("*.zip"):
        # Skip the outer archive itself if present in the scan
        if zip_file.samefile(root):
            continue
        target_dir = zip_file.with_suffix("")
        target_dir.mkdir(parents=True, exist_ok=True)
        with zipfile.ZipFile(zip_file, "r") as zf:
            members = zf.namelist()
            print(f"📦 Extracting inner {zip_file} -> {target_dir} ({len(members)} entries)")
            for member in tqdm(members, desc=f"Extract {zip_file.name}", unit="file"):
                zf.extract(member, target_dir)
        extracted_dirs.append(target_dir)
    return extracted_dirs


def ensure_target_dirs(base: Path) -> None:
    (base / "video" / "omnivore").mkdir(parents=True, exist_ok=True)
    (base / "video" / "slowfast").mkdir(parents=True, exist_ok=True)
    (base / "audio").mkdir(parents=True, exist_ok=True)


def decide_target(file_path: Path, target_root: Path) -> Path:
    lower_parts = [p.lower() for p in file_path.parts]
    name_lower = file_path.name.lower()

    if "omnivore" in lower_parts or "omnivore" in name_lower:
        return target_root / "video" / "omnivore" / file_path.name
    if "slowfast" in lower_parts or "slowfast" in name_lower:
        return target_root / "video" / "slowfast" / file_path.name
    if "audio" in lower_parts or name_lower.endswith("_audio.npz"):
        return target_root / "audio" / file_path.name
    # Fallback: treat as omnivore video
    return target_root / "video" / "omnivore" / file_path.name


def move_npz_files(source_root: Path, target_root: Path) -> int:
    ensure_target_dirs(target_root)
    moved = 0
    for npz in source_root.rglob("*.npz"):
        target = decide_target(npz, target_root)
        target.parent.mkdir(parents=True, exist_ok=True)
        if not target.exists():
            shutil.move(str(npz), str(target))
            moved += 1
    return moved


def verify(target_root: Path) -> None:
    print("\n✅ Verification:")
    for label, rel in [
        ("Video Omnivore", target_root / "video" / "omnivore"),
        ("Video SlowFast", target_root / "video" / "slowfast"),
        ("Audio", target_root / "audio"),
    ]:
        count = len(list(rel.glob("*.npz"))) if rel.exists() else 0
        status = "✓" if count > 0 else "✗"
        print(f"  {status} {label}: {count} files in {rel}")


def cleanup(temp_dir: Path) -> None:
    if temp_dir.exists():
        shutil.rmtree(temp_dir)
        print(f"🧹 Removed {temp_dir}")


def main():
    on_colab = mount_google_drive()

    outer_zip = Path("/content/drive/MyDrive/CaptainCook4D/features.zip" if on_colab else "./features.zip")
    if not outer_zip.exists():
        raise FileNotFoundError(f"Zip not found at {outer_zip}")

    temp_root = Path("./features_temp")
    target_root = Path("./content/code/data")

    # Step 1: Extract outer zip
    extract_zip(str(outer_zip), str(temp_root))

    # Step 2: Extract inner zips (omnivore.zip, slowfast.zip, etc.)
    extract_inner_zips(temp_root)

    # Step 3: Move npz files into expected layout
    moved = move_npz_files(temp_root, target_root)
    print(f"\n📂 Moved {moved} .npz files into {target_root}")

    # Step 4: Verify
    verify(target_root)

    # Step 5: Cleanup temp
    cleanup(temp_root)

    print("\nDone. You can now run training/eval with data under ./data")


if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Google Drive mounted at /content/drive
📦 Extracting /content/drive/MyDrive/CaptainCook4D/features.zip -> features_temp (2 entries)


Extract: 100%|██████████| 2/2 [00:07<00:00,  3.88s/file]


📦 Extracting inner features_temp/slowfast.zip -> features_temp/slowfast (385 entries)


Extract slowfast.zip: 100%|██████████| 385/385 [00:04<00:00, 95.21file/s]


📦 Extracting inner features_temp/omnivore.zip -> features_temp/omnivore (385 entries)


Extract omnivore.zip: 100%|██████████| 385/385 [00:14<00:00, 26.17file/s]



📂 Moved 768 .npz files into data

✅ Verification:
  ✓ Video Omnivore: 384 files in data/video/omnivore
  ✓ Video SlowFast: 384 files in data/video/slowfast
  ✗ Audio: 0 files in data/audio
🧹 Removed features_temp

Done. You can now run training/eval with data under ./data


In [10]:
!mkdir /content/code/checkpoints
!unzip /content/drive/MyDrive/CaptainCook4D/error_recognition_best.zip -d /content/code/checkpoints

mkdir: cannot create directory ‘/content/code/checkpoints’: File exists
Archive:  /content/drive/MyDrive/CaptainCook4D/error_recognition_best.zip
  inflating: /content/code/checkpoints/error_recognition_best/MLP/3dresnet/error_recognition_MLP_3dresnet_recordings_epoch_45.pt  
  inflating: /content/code/checkpoints/error_recognition_best/MLP/3dresnet/error_recognition_MLP_3dresnet_person_epoch_39.pt  
  inflating: /content/code/checkpoints/error_recognition_best/MLP/3dresnet/error_recognition_MLP_3dresnet_step_epoch_41.pt  
  inflating: /content/code/checkpoints/error_recognition_best/MLP/3dresnet/error_recognition_MLP_3dresnet_environment_epoch_11.pt  
  inflating: /content/code/checkpoints/error_recognition_best/MLP/imagebind/error_recognition_MLP_imagebind_audio_step_epoch_28.pt  
  inflating: /content/code/checkpoints/error_recognition_best/MLP/imagebind/error_recognition_MLP_imagebind_audio_recordings_epoch_2.pt  
  inflating: /content/code/checkpoints/error_recognition_best/MLP/im

In [8]:
!pip install torcheval

In [17]:
!mv /content/data /content/code/

In [18]:
%%bash

cd code
python -m core.evaluate --variant MLP --backbone omnivore \
  --ckpt /content/code/checkpoints/error_recognition_best/MLP/omnivore/error_recognition_MLP_omnivore_step_epoch_43.pt \
  --split step --threshold 0.6

Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json
----------------------------------------------------------------
test Sub Step Level Metrics: {'precision': 0.4096162736939436, 'recall': 0.2989708115404083, 'f1': 0.3456549302643129, 'accuracy': 0.6831416629277163, 'auc': np.float64(0.6541560352028618), 'pr_auc': tensor(0.3187)}
test Step Level Metrics: {'precision': 0.6607142857142857, 'recall': 0.14859437751004015, 'f1': 0.24262295081967214, 'accuracy': 0.7105263157894737, 'auc': np.float64(0.7573902166041213), 'pr_auc': tensor(0.3638)}
test Step Level Metrics per Category: {0: {'precision': 0.8085106382978723, 'recall': 0.05499276410998553, 'f1': 0.10298102981029811, 'accuracy': 0.17042606516290726, 'auc': np.float64(0.3165803319041887), 'pr_auc': tensor(0.8628)}, 2: {'precision': 0.14893617021276595, 'recall': 0.14285714285714285, 'f1': 0.14583333333333334, 'accuracy': 0.8972431077694235, 'auc': np.float64(0.6917522683305632), 'pr_auc': tensor(0.0

test Progress: 42347/798: 100%|██████████| 798/798 [00:10<00:00, 76.92it/s]


In [20]:
%%bash

cd code
python -m core.evaluate \
  --variant MLP \
  --backbone omnivore \
  --ckpt /content/code/checkpoints/error_recognition_best/MLP/omnivore/error_recognition_MLP_omnivore_step_epoch_43.pt \
  --split recordings \
  --threshold 0.4

Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json
----------------------------------------------------------------
test Sub Step Level Metrics: {'precision': 0.5234383954154728, 'recall': 0.36225906242563655, 'f1': 0.4281830114382149, 'accuracy': 0.6818466353677621, 'auc': np.float64(0.7017401173803638), 'pr_auc': tensor(0.3993)}
test Step Level Metrics: {'precision': 0.6183745583038869, 'recall': 0.7261410788381742, 'f1': 0.6679389312977099, 'accuracy': 0.7406855439642325, 'auc': np.float64(0.8147061661680981), 'pr_auc': tensor(0.5474)}
test Step Level Metrics per Category: {0: {'precision': 0.7546468401486989, 'recall': 0.3548951048951049, 'f1': 0.4827586206896552, 'accuracy': 0.35171385991058124, 'auc': np.float64(0.3052553507098961), 'pr_auc': tensor(0.8177)}, 2: {'precision': 0.12267657992565056, 'recall': 0.673469387755102, 'f1': 0.20754716981132076, 'accuracy': 0.624441132637854, 'auc': np.float64(0.7218649517684887), 'pr_auc': tensor(0.1065)},

test Progress: 38340/671: 100%|██████████| 671/671 [00:04<00:00, 150.54it/s]


In [22]:
%%bash

cd code
python -m core.evaluate --variant Transformer --backbone omnivore \
  --ckpt /content/code/checkpoints/error_recognition_best/Transformer/omnivore/error_recognition_Transformer_omnivore_step_epoch_9.pt \
  --split step --threshold 0.6

Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json
----------------------------------------------------------------
test Sub Step Level Metrics: {'precision': 0.4445452483556362, 'recall': 0.6613801248523705, 'f1': 0.5317056629365887, 'accuracy': 0.6738848088412402, 'auc': np.float64(0.7461755308526944), 'pr_auc': tensor(0.3888)}
test Step Level Metrics: {'precision': 0.5155709342560554, 'recall': 0.5983935742971888, 'f1': 0.5539033457249071, 'accuracy': 0.6992481203007519, 'auc': np.float64(0.7561832027563805), 'pr_auc': tensor(0.4338)}
test Step Level Metrics per Category: {0: {'precision': 0.778169014084507, 'recall': 0.31982633863965265, 'f1': 0.4533333333333333, 'accuracy': 0.33208020050125314, 'auc': np.float64(0.3025278277452426), 'pr_auc': tensor(0.8379)}, 2: {'precision': 0.09859154929577464, 'recall': 0.5714285714285714, 'f1': 0.16816816816816818, 'accuracy': 0.6528822055137845, 'auc': np.float64(0.6967385084875072), 'pr_auc': tensor(0.0827)}

test Progress: 42347/798: 100%|██████████| 798/798 [00:06<00:00, 131.33it/s]


In [25]:
%%bash

cd code
python -m core.evaluate \
  --variant Transformer \
  --backbone omnivore \
  --ckpt /content/code/checkpoints/error_recognition_best/Transformer/omnivore/error_recognition_Transformer_omnivore_recordings_epoch_31.pt \
  --split recordings \
  --threshold 0.4

Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json
----------------------------------------------------------------
test Sub Step Level Metrics: {'precision': 0.4491327720864185, 'recall': 0.35123344173871657, 'f1': 0.39419567346212053, 'accuracy': 0.645018257694314, 'auc': np.float64(0.6254427005929003), 'pr_auc': tensor(0.3711)}
test Step Level Metrics: {'precision': 0.46551724137931033, 'recall': 0.3360995850622407, 'f1': 0.39036144578313253, 'accuracy': 0.6229508196721312, 'auc': np.float64(0.6226768310334846), 'pr_auc': tensor(0.3949)}
test Step Level Metrics per Category: {0: {'precision': 0.7873563218390804, 'recall': 0.2395104895104895, 'f1': 0.3672922252010724, 'accuracy': 0.2965722801788376, 'auc': np.float64(0.39113159567705025), 'pr_auc': tensor(0.8369)}, 2: {'precision': 0.08620689655172414, 'recall': 0.30612244897959184, 'f1': 0.13452914798206278, 'accuracy': 0.7123695976154992, 'auc': np.float64(0.5750377321346545), 'pr_auc': tensor(0.07

test Progress: 38340/671: 100%|██████████| 671/671 [00:04<00:00, 135.47it/s]


In [ ]:
"""
NOTA:
il parametro --threshold consigliato nei repository ufficiali è ERRATO.
Se si vogliono ottenere ESATTAMENTE i risultati della tabella nel paper,
usare --threshold 0.5 e non 0.4 come specificato nei README ufficiali.
"""